In [10]:
import pandas as pd
from gurobipy import Model, GRB

In [11]:
# Load the data from the CSV file
costs_df = pd.read_csv('/Users/mahinbindra/Downloads/costs.csv')

In [12]:
# Initialize the model
model = Model("HealthLink Supplies Distribution")

In [13]:
# Define the decision variables
x = model.addVars(costs_df['Entry'], vtype=GRB.BINARY, name="x")  # Binary decision variable for site selection
s = model.addVars(costs_df['Entry'], vtype=GRB.CONTINUOUS, lb=0, name="s")  # Continuous decision variable for units stored

In [14]:
# Objective function: Minimize total cost
model.setObjective(sum(costs_df['Fixed'][i-1] * x[i] + costs_df['Variable'][i-1] * s[i] for i in costs_df['Entry']), GRB.MINIMIZE)

In [15]:
# Constraints
# 1. Capacity constraints
for i in costs_df['Entry']:
    model.addConstr(s[i] >= 175000 * x[i], name=f"min_capacity_{i}")
    model.addConstr(s[i] <= 375000 * x[i], name=f"max_capacity_{i}")

# 2. At least four locations among sites 6-16
model.addConstr(sum(x[i] for i in range(6, 17)) >= 4, name="location_requirement_6_16")

# 3. No more than 6 locations among the even-numbered sites
model.addConstr(sum(x[i] for i in costs_df['Entry'] if i % 2 == 0) <= 6, name="even_location_limit")

# 4. Conditional constraints for site selection
for j in [5, 6, 7]:
    model.addConstr(x[1] + x[2] <= 1 - x[j], name=f"conditional_sites_1_2_{j}")

# 5. Conditional constraints for location 19-22 affecting 24, 26, and 27
for j in [24, 26, 27]:
    model.addConstr(sum(x[i] for i in range(19, 23)) <= 1 - x[j], name=f"conditional_sites_19_22_{j}")

# 6. If any location from 1-5 is chosen, at least one odd site from 21-27 must be chosen
model.addConstr(sum(x[i] for i in range(1, 6)) <= sum(x[i] for i in costs_df['Entry'] if i >= 21 and i % 2 == 1), name="odd_sites_selection")

# 7. Equal number of locations in 1-14 and 15-27
model.addConstr(sum(x[i] for i in range(1, 15)) == sum(x[i] for i in range(15, 28)), name="equal_distribution_sites")

# 8. Equal sum of units in 1-9 and 19-27
model.addConstr(sum(s[i] for i in range(1, 10)) == sum(s[i] for i in range(19, 28)), name="equal_units_distribution")

<gurobi.Constr *Awaiting Model Update*>

In [16]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 65 rows, 54 columns and 210 nonzeros
Model fingerprint: 0x5f766475
Variable types: 27 continuous, 27 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+05]
  Objective range  [2e-01, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Presolve removed 18 rows and 9 columns
Presolve time: 0.00s
Presolved: 47 rows, 45 columns, 174 nonzeros
Variable types: 18 continuous, 27 integer (27 binary)
Found heuristic solution: objective 1.086250e+07

Root relaxation: objective 6.765500e+06, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6765500.0000 6765500.00  0.00%    

In [17]:
# Display the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
    print(f"Total Cost: {model.objVal}")
    for i in costs_df['Entry']:
        if x[i].X > 0.5:  # Only print chosen locations
            print(f"Location {i} selected with {s[i].X} units.")
else:
    print("Optimal solution not found.")

Optimal solution found.
Total Cost: 6765500.0
Location 11 selected with 175000.0 units.
Location 13 selected with 175000.0 units.
Location 15 selected with 175000.0 units.
Location 16 selected with 175000.0 units.
